In [1]:
import ast
import datetime
import hashlib
import json
import logging
import math
import re
import time
import uuid
import urllib3

import redis
import requests
import xmltodict
from pymongo import MongoClient
from bs4 import BeautifulSoup
from connection import get_in,ConnectionTask
from pprint import pprint
import conf

c=ConnectionTask()
urllib3.disable_warnings()



In [2]:
def ctrip_fetch(data):
    """酒店订单精简信息查询接口 D_HotelOrderMiniInfo
    """
    app_code = data['appCode'] if data.get('appCode') == 'oklife' else 'other'
    ping_url = 'http://{}/Hotel/D_HotelOrderMiniInfo.asmx'.format(conf.CTRIP_API_URL)
    ts = int(time.time())
    secret = hashlib.md5(conf.CTRIP_KEY[app_code].encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3} D_HotelOrderMiniInfo'.format(
        ts, conf.CTRIP_AID[app_code], secret, conf.CTRIP_SID[app_code]).encode('utf-8')).hexdigest().upper()
    request_xml = conf.CTRIP_ORDER_XML.format(conf.CTRIP_AID[app_code], conf.CTRIP_SID[app_code], ts, sign, data['partnerId'])
    encode_xml = request_xml.encode('utf-8')
    headers = {'Content-Type': 'text/xml; charset=utf-8',
               'Host': 'openapi.ctrip.com', 'Content-Length': str(len(encode_xml)),
               'SOAPAction': "http://ctrip.com/Request"}
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    xmlt_data = (json.loads((json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
    return xmlt_data
    resp_data = xmlt_data['Response']['HotelOrderMiniInfoResponse']
    return resp_data

In [10]:
data={'appCode': 'oklife', 'partnerId': '5378153199', 'orderId': 'ok203552196343118'}
ctrip_fetch(data)

{'Response': {'Header': {'AccessCount': '10000',
   'CurrentCount': '1',
   'RecentlyTime': '2018-03-01 09:48:56',
   'ReferenceID': '9a3f5b5f-a7de-405f-8a60-a3f710786b3d',
   'ResetTime': '2018-03-01 09:49:56',
   'ResultCode': 'Success',
   'ShouldRecordPerformanceTime': 'False',
   'Timestamp': '2018-03-01T09:48:56.7121820+08:00'},
  'HotelOrderMiniInfoResponse': {'AllianceId': '698282',
   'CheckInDate': '2018-02-08T00:00:00',
   'CheckOutDate': '2018-02-10T00:00:00',
   'FromOrderId': '0',
   'HotelId': '1685810',
   'HotelName': '长春吉祥饭店',
   'OrderDate': '2018-02-08T16:05:40',
   'OrderId': '5378153199',
   'OrderStatus': '已成交',
   'Ouid': None,
   'Price': '592.0000',
   'Quantity': '1',
   'Sid': '1235859'}}}

In [31]:
def token_fetch_and_update():
    """获取、更新携程access token,存储于radis中,失效后可再次获取
    """
    token_payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'KEY': conf.XIECHENG_TOKEN_KEY,
    }
    token = 'https://openserviceauth.ctrip.com/openserviceauth/authorize.ashx'
    token_result = requests.post(token, params=token_payload)
    Refresh_Token = token_result.json()['Refresh_Token']

    atoken_payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'refresh_token': Refresh_Token,
    }
    Atoken = 'https://openserviceauth.ctrip.com/openserviceauth/refresh.ashx'

    Atoken_result = requests.post(Atoken, params=atoken_payload)
    Access_Token = Atoken_result.json()['Access_Token']
    # 存贮token与过期时间字段
    c.dpprdb(0).hset('ctrip_token', 'access_token', Access_Token)
    c.dpprdb(0).hset('ctrip_token', 'expire_time', datetime.datetime.now() + datetime.timedelta(seconds=60 * 9))
    return ['xiecheng_access_token update is OK!']


In [39]:
def city_get_full_hotel_ids(city, page_index=1):
    """根据城市ID，查询酒店全量ID列表，并调用静态信息接口
    """
    Access_Token = c.dpprdb(0).hget('ctrip_token', 'access_token')
    print(Access_Token)
    # 从redis获取Access token
    payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'ICODE': conf.XIECHENG_ICODE,
        'UUID': conf.XIECHENG_UUID,
        'Token': Access_Token,
        'mode': 1,
        'format': 'json'
    }
    url_html = 'http://openservice.ctrip.com/OpenService/ServiceProxy.ashx'
    data = {
        "City": city,
        "PageIndex": str(page_index),
        "PageSize": "5000"
    }

    try:
        requests_result = requests.post(url_html, params=payload, data=data)
        resp_data = requests_result.json()
        return resp_data
        total = resp_data['Total']
        if int(total) == 0:
            return ['city', city, 'total 0']
        hotel_ids = resp_data['HotelIDs'].split(',')
        return hotel_ids
    except KeyError as e:
        # 如果报错，token过期，执行token_fetch函数更新token
        print(resp_data)
        if str(datetime.datetime.now()) > (c.dpprdb(0).hget('ctrip_token', 'expire_time')):
            token_fetch_and_update()
        # 接口问题，创建一个60s延迟的自身调用异步任务
#         city_get_full_hotel_ids.apply_async(args=[city, page_index], countdown=60, queue=conf.MQ_XIECHENG)
        return['city', city, 'EOFError and Retry']

#     if page_index * 5000 < total:
        # 执行下一页的参数查询
#         city_get_full_hotel_ids.apply_async(args=[city, page_index + 1], queue=conf.MQ_XIECHENG)
    for hotel_ids_index in range(len(hotel_ids) // 10 + 1):
        return hotel_ids[hotel_ids_index * 10:(hotel_ids_index + 1) * 10]
        # 切片酒店ID列表，调用静态信息接口
#         hotel_ids_message_fetch(args=[hotel_ids[hotel_ids_index * 10:(hotel_ids_index + 1) * 10]],
#                                             queue=conf.MQ_XIECHENG)  # async
        break  # test
    return [city, page_index, len(hotel_ids), 'is over']

In [48]:
hotel_ids = city_get_full_hotel_ids(1236)['HotelIDs'].split(',')

641983fae0b945f7972c1eda162ed6b7


In [56]:
hotel_ids_message_fetch(['6887689', '6887690', '6887691', '6887692', '6887693', '6887695', '6887697', '6887698', '6887700', '6887701'])

[{'AffiliationInfo': {'Awards': {'Award': [{'Provider': 'HotelStarLicence',
      'Rating': '0'},
     {'Provider': 'HotelStarRate', 'Rating': '0'},
     {'Provider': 'CtripStarRate', 'Rating': '0'},
     {'Provider': 'CtripUserRate', 'Rating': '0'},
     {'Provider': 'CtripCommRate', 'Rating': '0'}]}},
  'AreaID': '0',
  'AreaInfo': {'RefPoints': {'RefPoint': {'CategoryCode': '1002',
     'DescriptiveText': '驾车约181.66公里(约210分钟)',
     'Distance': '181.66',
     'Latitude': '23.884154',
     'Longitude': '100.088824',
     'Name': '市中心',
     'RefPointCategoryCode': '10004',
     'RefPointName': '市中心',
     'UnitOfMeasureCode': '2'}}},
  'BrandCode': '0',
  'ContactInfos': {'ContactInfo': {'Phones': {'Phone': [{'PhoneNumber': '15012098050',
       'PhoneTechType': '1'},
      {'PhoneNumber': '', 'PhoneTechType': '3'}]}}},
  'FacilityInfo': {'GuestRooms': None},
  'HotelCityCode': '1236',
  'HotelCode': '6887689',
  'HotelInfo': {'Address': {'AddressLine': '中缅路东段',
    'CityName': '临沧',

In [11]:
def hotel_ids_message_fetch(hotel_ids):
    """根据酒店ID，查询酒店静态信息,并返回数据进行清洗，最后调用价格计划接口(if rateplans is True)"""
    ping_url = 'http://{}/Hotel/OTA_HotelDescriptiveInfo.asmx'.format(
        conf.XIECHENG_API_URL)
    ts = int(time.time())
    secret = hashlib.md5(conf.XIECHENG_KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_Ping'.format(ts,
                                                     conf.XIECHENG_AID,
                                                     secret,
                                                     conf.XIECHENG_SID).encode('utf-8')).hexdigest().upper()
    request_xml = (conf.XIECHENG_MESSAGE_XML)
    hotel_desc = (conf.XIECHENG_MESSAGE_DESC)
    hotel_index = ''
    # 动态创建n个id的request body
    n = len(hotel_ids)
    if n == 0:
        return ['hotel_ids is []']
    for i in range(n):
        hotel_index += hotel_desc.replace('index', str(i))
    hotel_desc = hotel_index.format(hotel_ids)
    request_xml = request_xml.format(conf.XIECHENG_AID, conf.XIECHENG_SID, ts, sign, hotel_desc)
    encode_xml = request_xml.encode('utf-8')
    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'Host': 'openapi.ctrip.com',
        'Content-Length': str(len(encode_xml))
    }

    resp = requests.post(url=ping_url, data=encode_xml, headers=headers)
    if resp.status_code != 200:
        return [resp.status_code, 'requests err', hotel_ids]
    xmlt_data = xmltodict.parse(xmltodict.parse(resp.text)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    # 调用轮子xmltodict，将xml格式数据 转换为 dict格式数据，并删除‘@’字符
    xmlt_data = (json.loads((json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
#     return xmlt_data['Response']
    datas = xmlt_data['Response']['HotelResponse']['OTA_HotelDescriptiveInfoRS'][
        'HotelDescriptiveContents']['HotelDescriptiveContent']
    if not isinstance(datas, list):
        datas = [datas]
    return datas

In [30]:
hotel_ids_message_fetch(['13955695']) # 19:52 -21:18=8+60+18=86 /100  !! 69.7/min
# 16:25:26 -16:27:41

[{'AffiliationInfo': {'Awards': {'Award': [{'Provider': 'HotelStarLicence',
      'Rating': '0'},
     {'Provider': 'HotelStarRate', 'Rating': '0'},
     {'Provider': 'CtripStarRate', 'Rating': '0'},
     {'Provider': 'CtripUserRate', 'Rating': '0'},
     {'Provider': 'CtripCommRate', 'Rating': '0'},
     {'Provider': 'CommSurroundingRate', 'Rating': '0'},
     {'Provider': 'CommFacilityRate', 'Rating': '0'},
     {'Provider': 'CommCleanRate', 'Rating': '0'},
     {'Provider': 'CommServiceRate', 'Rating': '0'}]}},
  'AreaID': '256',
  'AreaInfo': {'RefPoints': {'RefPoint': [{'CategoryCode': '1002',
      'DescriptiveText': '驾车约2.13公里(约6分钟)',
      'Distance': '2.13',
      'Latitude': '23.0048160552979',
      'Longitude': '113.329811096191',
      'Name': '番禺长隆、广州南站附近',
      'RefPointCategoryCode': '10001',
      'RefPointName': '商业区',
      'UnitOfMeasureCode': '2'},
     {'CategoryCode': '1002',
      'DescriptiveText': '驾车约50.46公里(约96分钟)',
      'Distance': '50.46',
      'Latitud

In [73]:
hotel_ids_rateplans_fetch(['1361135'],pay_type=1)

{'HotelCode': '1361135',
 'RatePlan': [{'AnticipationCoefficient': '1',
   'Applicability': '10000000',
   'BookingRules': {'BookingRule': {'ChannelIDs': {'ChannelID': ['2',
       '6',
       '22']}}},
   'Description': {'Name': '豪华套房[含早]'},
   'InvoiceTargetType': '2',
   'IsCommissionable': 'false',
   'IsHasSpecialInvoice': 'true',
   'IsHourRoom': 'false',
   'IsSupportAnticipation': 'false',
   'Offers': None,
   'OtherDescription': '',
   'RatePlanCategory': '16',
   'RatePlanCode': '7654533',
   'RateReturn': 'false',
   'Rates': {'Rate': [{'BaseByGuestAmts': {'BaseByGuestAmt': {'AmountBeforeTax': '798.00',
        'CurrencyCode': 'CNY',
        'ListPrice': '798.00',
        'NumberOfGuests': '2',
        'TPA_Extensions': {'OtherCurrency': {'AmountPercentType': {'Amount': '798.00',
           'CurrencyCode': 'CNY'}}}}},
      'CancelPolicies': {'CancelPenalty': {'AmountPercent': {'Amount': '798.00',
         'CurrencyCode': 'CNY'},
        'End': '2018-3-7 0:00:00',
        '

In [72]:
def hotel_ids_rateplans_fetch(hotel_ids,pay_type=0):
    """ 查询酒店ID对应price日历，返回最低价格
    Args:
        pay_type(int): 0线付，1现付
    """
#     AID_FG: '263411',
#             SID_FG: '712773',
#             KEY_FG: '1F100349-B1C2-4C92-B6CD-69BE97EAF7F8',
    start = str(datetime.date.today())
    # 价格日历28天
    end = str(datetime.date.today() + datetime.timedelta(days=28))
    ping_url = 'http://{}/Hotel/OTA_HotelRatePlan.asmx'.format(conf.XIECHENG_API_URL)
    ts = int(time.time())
    secret = hashlib.md5(
        conf.XIECHENG_KEY.encode('utf8') if pay_type==0 else '1F100349-B1C2-4C92-B6CD-69BE97EAF7F8'.encode('utf8')
    ).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_HotelRatePlan'.format(ts,
                                                              conf.XIECHENG_AID if pay_type==0 else 263411,
                                                              secret,
                                                              conf.XIECHENG_SID if pay_type==0 else 712773
                                                             ).encode('utf-8')).hexdigest().upper()

    # 动态创建n个id 的request body
    request_xml = (conf.XIECHENG_PRICE_XML)
    rateplan = (conf.XIECHENG_PRICE_RATEPLAN)
    rateplans = ''
    n = len(hotel_ids)
    for index in range(n):
        rateplans += rateplan.replace('index', str(index))
    rateplans = rateplans.format(hotel_ids=hotel_ids, start=start, end=end)
    request_xml = request_xml.format(conf.XIECHENG_AID if pay_type==0 else 263411,
                                     conf.XIECHENG_SID if pay_type==0 else 712773,
                                     ts,
                                     sign,
                                     datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f+08:00'),
                                     rateplans=rateplans)

    encode_xml = request_xml.encode('utf-8')
    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'Host': 'openapi.ctrip.com',
        'Content-Length': str(len(encode_xml))
    }
    try:
        resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
        xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
        # xml 转 dict
        xmlt_data = (json.loads((json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
#         return xmlt_data['Response']['Header']['ResultCode']
        datas = xmlt_data['Response']['HotelResponse']['OTA_HotelRatePlanRS']['RatePlans']
        return datas
    except (KeyError, Exception) as e:
        return ['err and retry,errids:', hotel_ids]
    if not isinstance(datas, list):
        datas = [datas]
    return datas
    for data in datas:
        rateplans_get_price_and_insert(data)  # pylint: disable=E1120
    return [hotel_ids, 'rateplans fetch is OK']

In [187]:
hotel_ids_rateplans_fetch(['a82375311112223333332'])
# {'PrimaryLangID': 'zh',
#  'Success': None,
#  'TimeStamp': '2017-12-26T15:58:38.8747412+08:00',
#  'Version': '1.0',
#  'xmlns': 'http://www.opentravel.org/OTA/2003/05'}

'Success'

In [ ]:
# # 携程:主动更新酒店信息
# from path import Path
# import os
# a=Path('/root/dpp/')
# a.chdir()
# from ctrip_worker import hotel_ids_message_fetch
# hotels=[]
# count=0
# for database in c.static_ctrip.collection_names()[1:]:
#     print(database)
#     if 'jiudian' not in database:
#         continue
#     for item in c.static_ctrip[database].find({'rooms.tpa_Extensions':None},{'_id':0,'id':1}):
#         hotels.append(item['id'])
#         if len(hotels)==10:
#             hotel_ids_message_fetch.apply_async(args=[hotels],queue='dpp_ctrip_tasks')
#             count+=1
#             print(count,end='\r')
#             hotels=[]
#         else:
#             continue
# if hotels:
#     print('over',hotels)
#     hotel_ids_message_fetch.apply_async(args=[hotels],queue='dpp_ctrip_tasks')

In [ ]:
from path import Path
import os
a=Path('/root/dpp/')
a.chdir()
from ctrip_worker import hotel_ids_message_fetch,jiudian_hotels_sub_library
# hotel_ids_message_fetch.apply_async(args=[['1700306']],queue='dpp_ctrip_tasks')
jiudian_hotels_sub_library.apply_async(args=['706587'],queue='dpp_ctrip_tasks')

In [ ]:
for item in c.static.jiudian_hotels.find(
    {
        'cityCode':{
            '$in':['1','2','30','32']
        }
    },
    {
        'id':1,
        '_id':0
    }
):
    jiudian_hotels_sub_library.apply_async(args=[item['id']],queue='dpp_ctrip_tasks')
    

In [ ]:
# tag(list)
rdb=c.srdb_new(0)
count=0
err_data=[]
for collection in c.dmdb_ctrip.collection_names():
    if collection in ['jiudian_hotels_sort','system.indexes']:
        continue
    for clean_datas in c.dmdb_ctrip[collection].find({},
                                                    {'_id':0,
                                                     'id':1,
                                                     'areaId':1,
                                                     'zoneCode':1,
                                                     'name':1,
                                                     'brandCode':1}):
        try:
            name=clean_datas['name']
            tag=jieba_lcut_for_search(name)
            tag.add(name)
            sentence=name
            district_name = rdb.hget('ctrip_map:district', clean_datas['areaId'])
            brand_name=rdb.hget('ctrip_map:brand',clean_datas['brandCode'])
            if not brand_name:
                continue

            if clean_datas['zoneCode']:
                zone_list = [zone for zone in rdb.hmget('ctrip_map:zone', clean_datas['zoneCode']) if zone]
                tag=tag.union(set([item for item in punctuation_replace(','.join(zone_list)).split(',') if item]))
                sentence+=' '+str(zone_list)
            if district_name:
                tag.add(district_name)
                sentence+=' '+district_name
            if brand_name:
                tag.add(brand_name)
                sentence+=' '+brand_name
            tag=tag.union(jieba_analyse_extract_tags(sentence))
            count+=1
            c.dmdb_ctrip[collection].update_one({'id':clean_datas['id']},{'$set':{'tag':list(tag)}})
            c.dmdb_ctrip['jiudian_hotels_sort'].update_one({'id':clean_datas['id']},{'$set':{'tag':list(tag)}})
#             print(collection,clean_datas['id'],brand_name,tag,count,end='\n')
            print(count,end='\r')
#             break
        except:
            err_data.append(clean_datas)
#     break

In [ ]:
c.static_ctrip['jiudian_hotels_sort'].count({'brandCode':{'$in':brand_list}})

In [ ]:
brand_list=list((rdb.hgetall('ctrip_map:brand')).keys())

In [ ]:
import jieba.analyse,jieba
import zhon.hanzi
import string

def jieba_analyse_extract_tags(sentence):
    return set(jieba.analyse.extract_tags(sentence))

def jieba_lcut_for_search(sentence):
    sentence=punctuation_replace(sentence)
    tag=set()
    resp=jieba.lcut(sentence)
    for i in range(1,len(resp)+1):
        if ',' in resp[i-1]:
            break
        tag.add(''.join(resp[:i]))
    return tag

def punctuation_replace(sentence,replace=','):
    punctuation=string.punctuation+zhon.hanzi.punctuation
    trantab=str.maketrans(punctuation, replace*len(punctuation))
    return sentence.translate(trantab).strip(replace)

In [ ]:
jieba_lcut_for_search('星程酒店(南通体育会展中心店)')

In [ ]:
# 提取ctrip各城市不同地表分类统计
ref_points,count={},0
for item in c.static.jiudian_hotels.find({},{'_id':0,'cityCode':1,'refPoints':1},no_cursor_timeout=True):
    try:
        ref_points.setdefault(item['cityCode'],{})
        count+=1
        if not item.get('refPoints'):
            continue
        if not isinstance(item['refPoints'],list):
            item['refPoints']=[item['refPoints']]
        for ref_point in item['refPoints']:
            ref_points[item['cityCode']].setdefault(ref_point['RefPointName'],[])
            ref_points[item['cityCode']].setdefault('大学',[])
            ref_points[item['cityCode']].setdefault('医院',[])
            if ref_point.get('Name'):
                if '医院' in ref_point['Name']:
                    ref_points[item['cityCode']]['医院'].append(ref_point['Name'])
                elif '大学' in ref_point['Name']:
                    ref_points[item['cityCode']]['大学'].append(ref_point['Name'])
                else:
                    ref_points[item['cityCode']][ref_point['RefPointName']].append(ref_point['Name'])
        print(f">>>cityCode:{item['cityCode']},count:{count}<<<",end='\r')
    except:
        pprint(item)

In [ ]:
city_category_name={}
for city_code,value in ref_points.items():
    city_category_name[city_code]={}
    for category,name_list in value.items():
        city_category_name[city_code][category]=[item[0] for item in Counter(name_list).most_common(20)]
#     pprint(city_category_name)

In [ ]:
with open('ctrip_refPoints.json','w') as f:
    f.write(json.dumps(city_category_name,indent=2,ensure_ascii=False))

In [ ]:
city_category_name

In [ ]:
from collections import Counter
a=Counter([1,2,3,2,3,4,5,6,6,3,2,413,13,2])
# a.update([2])
[item[0] for item in a.most_common(2)]

In [ ]:
# dpp.boluome.com:ctrip.tag.decomposition:
count=0
for item in c.static.jiudian_hotels.find({'cityCode':'2'},{'tag':1},no_cursor_timeout=True):
    for tag in item['tag']:
        tag_lcut=sorted(list(jieba_lcut_for_search(tag)))[:-1]
        if not tag_lcut:
            continue
        for index,lcut in enumerate(tag_lcut):
            c.srdb_new(0).zincrby(
                'dpp.boluome.com:ctrip.tag.decomposition:{}'.format(lcut),
                tag,float(index+1))
    count+=1
    print(count,end='\r')

In [ ]:
sorted(list(jieba_lcut_for_search('北京不夜谷老孟农家院')))

In [203]:
c.static_ctrip.jiudian_hotels_30.count({}) # 2809

2140

In [204]:
c.static_ctrip.jiudian_hotels_30.count({'__v':10}) #2809

2140

In [208]:
c.static_ctrip.jiudian_hotels_30.count({'__v':10,'price':{'$exists':True},'price':{'$ne':-1}}) # 5447

4286

In [209]:
c.static_ctrip.jiudian_hotels_30.count({'__v':10}) # 5483

4441

In [205]:
c.static_ctrip.jiudian_hotels_30.update_many({},{'$rename':{'__v':'v'}})

In [212]:
city_code_list=list(c.srdb_new(0).zrangebyscore('ctrip_city_map', 0, 6))

In [218]:
city_code_list.remove('30')

In [235]:
from connection import ConnectionTask
c = ConnectionTask()
from path import Path
import os
a=Path('/root/dpp/')
a.chdir()
from ctrip_worker import *
print('ok')
count=0
hotel_list = []
c.s
for city_code in c.static_cluster_ctrip.jiudian_hotels_sort.distinct("cityCode"):
#     city_code = 1236
    count+=1
    print(count,end='\r')
    for item in c.static_cluster_ctrip.jiudian_hotels_sort.find({'cityCode':city_code},{'id':1,'_id':0}):
        hotel_list.append(item['id'])
        if len(hotel_list) == 10:
            search_hotel_lowprice.apply_async(args=[city_code,hotel_ids], queue='dpp_ctrip_tasks')
            hotel_list = []
    if hotel_list:
        search_hotel_lowprice.apply_async(args=[city_code,hotel_ids], queue='dpp_ctrip_tasks')
        hotel_list = []
    break

In [270]:
count=0
city_code_list = c.static_cluster_ctrip['jiudian_hotels_sort'].distinct("cityCode")
for city_code in city_code_list:
    for item in c.static_cluster_ctrip['jiudian_hotels_sort'].find(
        {'cityCode': city_code,'price':{'$exists':False}}, 
        {'id': 1, '_id': 0}):
        count+=1
        print(count,end='\r')
        hotel_list.append(item['id'])
        if len(hotel_list) == 20:
            resp = search_hotel_lowprice(city_code, hotel_list)
            hotel_list = []
            print(resp)
            break
        break
#             for hotel_data in get_in(resp,'HotelDataLists',[]):
#                 hotel_id = str(get_in(hotel_data, 'HotelStatusEntity.Hotel'))
#                 lowprice = get_in(hotel_data, 'HotelStatusEntity.MinPrice', default=-1)
#                 if hotel_id:
#                     print(hotel_id)
            
#     if hotel_list:
#         resp = search_hotel_lowprice(city_code, hotel_list)
#         for hotel_data in get_in(resp,'HotelDataLists',[]):
#             lowprice = get_in(hotel_data, 'HotelStatusEntity.MinPrice', default=-1)
#             if hotel_id:
#                 print(hotel_id)
#         hotel_list = []

cityCode:1025,hotelList:7927392,12189428,473528,1013801,781322,931812,1014748,1183510,982460,824897,1082010,1012409,967881,910731,1887095,1106677,1426180,855686,799796,1453221,hotelIdMapDatabase is None
cityCode:1043,hotelList:1113366,4817637,1192363,4833826,997155,2191884,993655,939761,4330394,1233216,820002,814868,795150,1180420,1184934,1063946,854551,1842853,1093577,906288,hotelIdMapDatabase is None
cityCode:1070,hotelList:1193997,4843345,1895659,1692022,1396795,811611,10937066,842613,971528,2025690,902493,1113288,1334409,1451423,1388237,1353295,11683804,899900,1618138,1700218,hotelIdMapDatabase is None
cityCode:1094,hotelList:797085,1600460,837336,797138,1289551,2130543,1312842,891330,918681,1307668,1856378,820020,2588585,1337853,968661,1763187,907762,6794868,1047322,855786,hotelIdMapDatabase is None
cityCode:1118,hotelList:1085913,965959,12296847,814974,814807,1182496,807437,853759,6843940,899826,1463611,859815,854013,805638,1192546,1617151,6480300,1886270,946710,2116468,hotelIdMa

cityCode:21323,hotelList:4834476,6777232,1468701,2159328,1986175,4834603,1499517,4820285,1250066,6819524,8455687,2705453,4842957,2289556,6762042,1199202,13706948,13877822,6728185,11419984,hotelIdMapDatabase is None
cityCode:21349,hotelList:11421270,11406685,6780347,6727683,6811627,4842259,5077878,11420429,834965,11815899,8547757,1196350,5083089,11415613,5375375,1608183,12480708,968156,6873636,5078413,hotelIdMapDatabase is None
cityCode:21375,hotelList:2258912,5315745,11409154,11910368,11499366,1434989,11909738,5373960,2277523,1120604,4835490,3630714,8564088,4819351,2321773,11815979,2310624,5079990,1644355,13967698,hotelIdMapDatabase is None
cityCode:21399,hotelList:11502409,1460683,6835311,1856412,11419946,1639599,4337236,4817506,6829896,852066,6803771,1129016,4824782,1702489,6852127,4746351,1108614,4741226,4822600,2314662,hotelIdMapDatabase is None
cityCode:21421,hotelList:5374997,6856073,5802935,4832282,2314836,4827784,4830675,6812005,1505782,4817918,6775759,1012490,4745298,1321689,5

cityCode:378,hotelList:1012120,809777,13967688,817369,1072833,796692,2136296,2156601,4831441,1340577,11817941,13953863,3003365,906507,6792663,13952767,471604,817215,4829099,1626142,hotelIdMapDatabase is None
cityCode:3910,hotelList:13471788,841491,2120453,890687,1197754,917380,840643,14153142,4831165,11467121,1952508,1097721,6724964,899622,1251357,2015634,1107274,4832956,820452,4824824,hotelIdMapDatabase is None
cityCode:3933,hotelList:11419023,6723520,2296076,6772664,8572223,11811973,899985,971044,11411983,848418,5245437,6789382,6834765,2271435,2502614,4331787,6827905,11297466,12192827,910850,hotelIdMapDatabase is None
cityCode:3970,hotelList:6810327,5382127,11412066,1099303,6800783,6765931,1215823,11420729,4741279,8571559,1231402,1436478,6509146,6790568,1293046,1067667,6857673,6827874,67723,1747890,hotelIdMapDatabase is None
cityCode:404,hotelList:963562,4815713,808179,4338773,1300491,5244949,6739210,4745534,11388073,6978998,6798497,903408,1978352,447329,13952015,1349478,6794611,6769

In [278]:
city_code,hotel_list
search_hotel_lowprice(city_code, hotel_list)

{'AllHotelID': '6746330',
 'HotelDataLists': [{'HotelStatusEntity': {'Currency_minPrice': 'RMB',
    'Hotel': 6746330,
    'MinPrice': 110,
    'MinPriceInfoEntity': {'MinPrice_AfterTax': {'MinPrice_CashBack': {'MinPrice': 110,
       'MinPrice_OriginalCurrency': 110,
       'MinPrice_RoomID': 0,
       'OriginalCurrency': 'RMB'}}},
    'MinPriceRoom': 75222128,
    'MinPrice_MyCurrency': 110}}],
 'NumEntity': {'CityCount': [{'City': 996,
    'CityEName': 'Gutian',
    'CityName': '古田',
    'CityNum': 1}]},
 'ResponseStatus': {'Ack': 'Success',
  'Errors': [],
  'Extension': [],
  'Timestamp': '2018-03-09T09:50:31.5076547+08:00'}}

In [286]:
search_hotel_lowprice('30', '1549637,1595181,1596231,1599182,1596049,1598159,1799001,1758471,1598808,1795280,1600185,1847462,1604027,1596942,1740590,1759734,1759984,1759887,1851047,1603097')

{'AllHotelID': '1596942,1604027,1799001,1847462,1740590,1598808,1758471,1599182,1600185,1598159,1795280,1851047,1596049,1759887,1596231,1603097,1595181',
 'HotelDataLists': [{'HotelStatusEntity': {'Currency_minPrice': 'RMB',
    'Hotel': 1851047,
    'MinPrice': 248,
    'MinPriceInfoEntity': {'MinPrice_AfterTax': {'MinPrice_CashBack': {'MinPrice': 248,
       'MinPrice_OriginalCurrency': 248,
       'MinPrice_RoomID': 0,
       'OriginalCurrency': 'RMB'}}},
    'MinPriceRoom': 74530099,
    'MinPrice_MyCurrency': 248}}],
 'NumEntity': {'CityCount': [{'City': 30,
    'CityEName': 'Shenzhen',
    'CityName': '深圳',
    'CityNum': 5}]},
 'ResponseStatus': {'Ack': 'Success',
  'Errors': [],
  'Extension': [],
  'Timestamp': '2018-03-09T14:09:04.3389130+08:00'}}

In [292]:
search_hotel_lowprice('30', '1549637')

{'AllHotelID': '',
 'HotelDataLists': [],
 'NumEntity': {'CityCount': []},
 'ResponseStatus': {'Ack': 'Success',
  'Errors': [],
  'Extension': [],
  'Timestamp': '2018-03-09T15:56:10.1685524+08:00'}}

In [275]:
# 1.4 最低价查询接口 SearchHotelLowPrice(透明代理接口)
# 功能:获取指定城市下各酒店的最低价。酒店列表页的起价常用到此接口
# 请求体示例:
def search_hotel_lowprice(city_code,hotel_list):
    if isinstance(hotel_list, list):
        hotel_list = ",".join(hotel_list)
    body={
        "SearchTypeEntity": {# <!--SearchType:查询类型;WirelessSearch，国内酒店;OnLineIntlGTASearch，海外酒店-->
            "SearchType": "WirelessSearch", # <!—分页参数，每页包含酒店数量-->
            "HotelCount": 20, # <!—页码-->
            "PageIndex": 1
        },
        "PublicSearchParameter": {
            "City": city_code,
            "CheckInDate": "2018-04-09", 
            "CheckOutDate": "2018-04-11", 
            "HotelList": hotel_list,
            "FilterRoomByPerson": "2,999",
            # <!—可选项，请求单人或多人最低价，1 代表单人价，2,999 代表 2 人及以上--> 
            "OnlyPPPrice":False, # <!—可选项，值为 true 只返回预付的最低价房型;false 则不作筛选处理 -> 
            "OnlyFGPrice":False, # <!—可选项，值为 true 只返回到付的最低价房型，false 则不作筛选出来-> },
    #         "FacilityEntity":{
    #             "IsJustifyConfirm":"T" # <!—可选项，值为 T 只返回可立即确认房型，值为 F 则不返回可立即确认房型，值为空则不筛选-->
    #         }
        }
    }
    url_html = 'http://openservice.ctrip.com/OpenService/ServiceProxy.ashx'
    token_fetch_and_update()
    Access_Token = c.dpprdb(0).hget('ctrip_token', 'access_token')
    payload = {
        'AID': conf.XIECHENG_AID,
        'SID': conf.XIECHENG_SID,
        'ICODE':'ea2570c91b3c4d19ab63fc6d97f8d62d',
        'Token': Access_Token,
        'mode': 1,
        'format': 'json'
    }
    requests_result = requests.post(url_html, params=payload, json=body)
    response = requests_result
    return response.json()

In [29]:
count = 0
for collection in c.static_cluster_ctrip.collection_names():
    if collection in ['jiudian_hotels_sort']:
        continue
    print(collection)
    for item in c.static_cluster_ctrip[collection].find({},{'_id':0,'images':1,'id':1}):
        c.static_cluster_ctrip['jiudian_hotels_sort'].update_one({'id':item['id']},{'$set':item})
        count += 1
        print(count,end='\r')

jiudian_hotels_7
jiudian_hotels_578
jiudian_hotels_375
jiudian_hotels_32
jiudian_hotels_38
jiudian_hotels_6
jiudian_hotels_491
jiudian_hotels_477
jiudian_hotels_376
jiudian_hotels_28
jiudian_hotels_17
jiudian_hotels_3
jiudian_hotels_223
jiudian_hotels_14
jiudian_hotels_42
jiudian_hotels_3t
jiudian_hotels_451
jiudian_hotels_31
jiudian_hotels_533
jiudian_hotels_82
jiudian_hotels_5t
jiudian_hotels_308
jiudian_hotels_213
jiudian_hotels_25
jiudian_hotels_406
jiudian_hotels_251
jiudian_hotels_559
jiudian_hotels_6t
jiudian_hotels_258
jiudian_hotels_144
jiudian_hotels_12
jiudian_hotels_5
jiudian_hotels_105
jiudian_hotels_158
jiudian_hotels_299
jiudian_hotels_428
jiudian_hotels_4t
jiudian_hotels_1
jiudian_hotels_553


In [22]:
count = 0
for collection in c.static_cluster_ctrip.collection_names():
    if collection in ['jiudian_hotels_sort','jiudian_hotels_7','jiudian_hotels_578']:
        continue
    print(collection)
    for item in c.static_cluster_ctrip[collection].find({},{'_id':0,'refPoints':1,'id':1}):
        refPoints = item['refPoints']
        if not refPoints:
            continue
        if isinstance(refPoints,dict):
            refPoints = [refPoints]
        for ref_item in refPoints:
            if 'Distance' in ref_item:
                ref_item['Distance'] = float(ref_item['Distance'])
        c.static_cluster_ctrip[collection].update_one(
            {'id':item['id']},
            {'$set':{'refPoints':refPoints}}
        )
        c.static_cluster_ctrip['jiudian_hotels_sort'].update_one(
            {'id':item['id']},
            {'$set':{'refPoints':refPoints}}
        )
        count += 1
        print(count,end='\r')

jiudian_hotels_375
jiudian_hotels_32
jiudian_hotels_38
jiudian_hotels_6
jiudian_hotels_491
jiudian_hotels_477
jiudian_hotels_376
jiudian_hotels_28
jiudian_hotels_17
jiudian_hotels_3
jiudian_hotels_223
jiudian_hotels_14
jiudian_hotels_42
jiudian_hotels_3t
jiudian_hotels_451
jiudian_hotels_31
jiudian_hotels_533
jiudian_hotels_82
jiudian_hotels_5t
jiudian_hotels_308
jiudian_hotels_213
jiudian_hotels_25
jiudian_hotels_406
jiudian_hotels_251
jiudian_hotels_559
jiudian_hotels_6t
jiudian_hotels_258
jiudian_hotels_144
jiudian_hotels_13
jiudian_hotels_206
jiudian_hotels_34
jiudian_hotels_39
jiudian_hotels_4
jiudian_hotels_512
jiudian_hotels_100
jiudian_hotels_571
jiudian_hotels_380
jiudian_hotels_30
jiudian_hotels_2
jiudian_hotels_22
jiudian_hotels_10
jiudian_hotels_278
jiudian_hotels_12
jiudian_hotels_5
jiudian_hotels_105
jiudian_hotels_158
jiudian_hotels_299
jiudian_hotels_428
jiudian_hotels_4t
jiudian_hotels_1
jiudian_hotels_553


In [37]:
hotel_id_list = c.static_cluster_ctrip['jiudian_hotels_sort'].distinct('id',{'name':{'$exists':False}})


In [38]:
for i in range(len(hotel_id_list)//10+1):
    print(i,hotel_id_list[i*10:i*10+10],end='\r')
    break
    